In [1]:
%config Completer.use_jedi = False

In [2]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [3]:
import pandas as pd

In [4]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [5]:
df = pd.read_csv('./datalab/googleplaystore.csv', nrows= 2000)

In [6]:
# df.head(8)

In [7]:
# df.info()

In [8]:
# df.isna().sum()

In [9]:
df.columns = [each.lower() for each in df.columns]

In [10]:
df['rating'].fillna(0, inplace=True)

In [11]:
df = df.drop(axis=0, columns=['current ver', 'android ver', 'app', 'genres', 'last updated'])

In [12]:
# df.columns = [each.replace(" ","_") if(len(each.split())>1) else each for each in df.columns]

In [13]:
df['size'] = [ each.replace('M','') if 'M' in str(each) else each for each in df['size']]

In [14]:
df['size'] = [ each.replace('k','') if 'k' in str(each) else each for each in df['size']]

In [15]:
df['size'] = [ each.replace('Varies with device','0') if 'Varies with device' in str(each) else each for each in df['size']]

In [16]:
df['installs'] = df['installs'].apply(lambda x: x.replace('+',"") if '+' in x else x)

In [17]:
df['installs'] = df['installs'].apply(lambda x: x.replace(',',"") if ',' in x else x)

In [18]:
df['installs'] = df.installs.astype(float)

In [19]:
df['size'] = df['size'].astype(float)

In [20]:
# df.info(memory_usage='deep')

In [21]:
# df.installs = df.installs.astype(int)

In [22]:
# df.reviews = df.reviews.astype(int)

In [23]:
df.price = df['price'].apply(lambda x : x.replace('$', '') if '$' in str(x) else str(x) )

In [24]:
df.price = df.price.astype(float)

In [25]:
# df['category'].unique()

In [26]:
# df['type'].unique()

In [27]:
# df['content rating'].unique()

In [28]:
df['type'] = df.type.astype('category')

In [29]:
df['category'] = df['category'].astype('category')

In [30]:
df['content rating'] = df['content rating'].astype('category')

In [31]:
df.category = df.category.cat.codes

In [32]:
df.type = df.type.cat.codes

In [33]:
df['content rating'] = df['content rating'].cat.codes

In [34]:
df.to_csv('./datalab/googleplaystoreml.csv', index=False, header=True)

In [35]:
# df.describe()

In [36]:
# corr = df.corr()
# ax = sns.heatmap(
#     corr, 
#     vmin=-1, vmax=1, center=0,
#     cmap=sns.diverging_palette(20, 220, n=200),
#     square=True
# )
# ax.set_xticklabels(
#     ax.get_xticklabels(),
#     rotation=90,
#     horizontalalignment='right'
# );

In [37]:
df.columns

Index(['category', 'rating', 'reviews', 'size', 'installs', 'type', 'price', 'content rating'], dtype='object')

In [38]:
model_ml = df[['category', 'type', 'price', 'size', 'installs', 'content rating', 'rating', 'reviews']]

In [39]:
# model_ml.head()

In [40]:
dummy_category = pd.get_dummies(model_ml['category'])

In [41]:
model_ml = pd.concat([model_ml, dummy_category], axis=1)

In [42]:
# model_ml.head().reset_index()

In [43]:
dummy_category = pd.get_dummies(model_ml['type'])
model_ml = pd.concat([model_ml, dummy_category], axis=1)

In [44]:
dummy_category = pd.get_dummies(model_ml['content rating'])
model_ml = pd.concat([model_ml, dummy_category], axis=1)

In [45]:
corr = model_ml.corr()

In [46]:
import seaborn as sns

In [47]:
model_ml = model_ml[['ART_AND_DESIGN', 'AUTO_AND_VEHICLES', 'BEAUTY', 'BOOKS_AND_REFERENCE', 'BUSINESS', 'COMICS', 'COMMUNICATION', 'DATING', 'EDUCATION', 'Paid', 'Adults only 18+', 'Everyone', 'Everyone 10+', 'Mature 17+', 'Teen', 'reviews','rating']]

KeyError: "['BUSINESS', 'COMMUNICATION', 'EDUCATION', 'DATING', 'COMICS', 'Mature 17+', 'ART_AND_DESIGN', 'Everyone', 'BOOKS_AND_REFERENCE', 'Teen', 'AUTO_AND_VEHICLES', 'BEAUTY', 'Adults only 18+', 'Paid', 'Everyone 10+'] not in index"

In [ ]:
X= model_ml.drop(['rating'], axis=1)

In [ ]:
# cor = model_ml.corr()
# ax = sns.heatmap(
#     corr, 
#     vmin=-1, vmax=1, center=0,
#     cmap=sns.diverging_palette(20, 220, n=200),
#     square=True
# )
# ax.set_xticklabels(
#     ax.get_xticklabels(),
#     rotation=90,
#     horizontalalignment='right'
# );

In [ ]:
Y = model_ml['rating']

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [ ]:
Y = Y.astype(int)

In [ ]:
# label = LabelEncoder()
# Y = label.fit_transform(Y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.25, random_state=0)

In [ ]:
print("Shape of X_train", X_train.shape)
print("Shape of X_test", X_test.shape)
print("Shape of Y_train", y_train.shape)
print("Shape of Y_test", y_test.shape)

In [ ]:
clf = DecisionTreeClassifier(criterion="entropy", random_state=0)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_test, y_pred))

In [ ]:
import numpy as np
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
actual_predcition = pd.DataFrame({"Actual": y_test.values.flatten(),"Predicted": y_pred.flatten()})
df1 = actual_predcition.head(45)
df1.plot(kind='bar',figsize=(30,14))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()


<h2>Logistics Regression Example </h2>

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X=StandardScaler()
X_train=sc_X.fit_transform(X_train)
X_test=sc_X.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr_c=LogisticRegression(random_state=0)
lr_c.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
lr_pred=lr_c.predict(X_test)
lr_cm=confusion_matrix(y_test,lr_pred)
lr_ac=accuracy_score(y_test, lr_pred)
print('LogisticRegression_accuracy:',lr_ac)

In [ ]:
actual_predcition = pd.DataFrame({"Actual": y_test.values.flatten(),"Predicted": lr_pred.flatten()})
df1 = actual_predcition.head(105)
df1.plot(kind='bar',figsize=(30,14))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

In [ ]:
# pred_value = pd.DataFrame({'Actual':y_test.flatten() , 'Prediction':y_pred.flatten()})

In [ ]:
# pred_value.head(20)

In [ ]:
# import numpy as np

In [ ]:
# print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
# print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
# print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
# from sklearn.linear_model import LinearRegression

In [ ]:
# regressor = LinearRegression().fit(X_train, y_train)

In [ ]:
# print(regressor.intercept_)
# print(regressor.coef_)

In [ ]:
# y_pred = regressor.predict(X_test)

In [ ]:
# pred_value = pd.DataFrame({'Actual':y_test.flatten() , 'Prediction':y_pred.flatten()})

In [ ]:
# pred_value.head(20)

In [ ]:
from sklearn.svm import SVC

In [ ]:
classifier = SVC(kernel='rbf', random_state = 0)
classifier.fit(X_train,y_train)

In [ ]:
Y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test,Y_pred)
accuracy = float(cm.diagonal().sum())/len(y_test)
print("\nAccuracy Of SVM For The Given Dataset : ", accuracy)